In [ ]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
sys.path.insert(0, "/notebooks")

from sentence_transformers import SentenceTransformer, util
from nebula3_database.database.milvus import MilvusAPI
mfilled = MilvusAPI('milvus', 'atomic_isfilled', 'video_cskg', 768)
matloc = MilvusAPI('milvus', 'atomic_atlocation', 'video_cskg', 768)
mafter = MilvusAPI('milvus', 'atomic_isafter', 'video_cskg', 768)
mbefore= MilvusAPI('milvus', 'atomic_isbefore', 'video_cskg', 768)
seq= MilvusAPI('milvus', 'atomic_sequence_rev', 'video_cskg', 768)
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
import numpy as np

In [ ]:
import glob
import csv
kg = []
relations = ['oWant', 'AtLocation', 'oReact', 'xWant', 'ObjectUse', 'HinderedBy', 'oEffect', 'xIntent', 'isFilledBy', 'xAttr', 
'HasSubEvent', 'isAfter', 'HasProperty', 'xReact', 'Desires', 'xEffect', 'xNeed', 'Causes', 'isBefore', 'CapableOf', 'xReason', 
'NotDesires', 'MadeUpOf']
print(len(relations))

print(relations)
files = glob.glob('atomic2020_data/*.tsv')
for file in files:
    with open(file) as in_file:
        tsv_file = csv.reader(in_file, delimiter="\t")
        for i, line in enumerate(tsv_file):
            sentence = ""
            #kg['node'] = line
            head = line[0]
            relation = line[1]
            tail = line[2]
            #print(tail)
            if relation == 'isFilledBy':
                sentence = head.replace('___', tail)
                sentence = sentence.replace('___', '')
            if relation == 'AtLocation':
                sentence = head + ' located at ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'oWant' and tail != 'none':
                sentence = head + ' he want ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xWant' and tail != 'none':
                sentence = head + ' he want ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'oReact' and tail != 'none':
                sentence = head + ' and then ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'ObjectUse' and tail != 'none':
                sentence = head + ' used for ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'HinderedBy' and tail != 'none':
                sentence = head + ', this would not happen if ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'oEffect' and tail != 'none':
                sentence = head + ' so he ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xIntent' and tail != 'none':
                sentence = head + ' in order ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xAttr' and tail != 'none':
                sentence = head + ' so he is ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'HasSubEvent' and tail != 'none':
                sentence = head + ' and ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'isAfter' and tail != 'none':
                sentence = head + ', before ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'HasProperty' and tail != 'none':
                sentence = head + ' is ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xReact' and tail != 'none':
                sentence = head + ' and then ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'Desires' and tail != 'none':
                sentence = head + ' desires ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xEffect' and tail != 'none':
                sentence = head + ' so he ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xNeed' and tail != 'none':
                sentence = head + ', he need ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'Causes' and tail != 'none':
                sentence = head + ' causes ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'isBefore' and tail != 'none':
                sentence = head + ' and then ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'CapableOf' and tail != 'none':
                sentence = head + ' is capable of ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'NotDesires' and tail != 'none':
                sentence = head + ' does not desire ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'MadeUpOf' and tail != 'none':
                sentence = head + ' is made up from ' + tail
                sentence = sentence.replace('___', '')
            kg.append({'entity': line, 'sentence': sentence})

In [ ]:
def get_vc(mdb):
        metadata = []
        query = 'FOR doc IN vcomet_kg LIMIT 100 RETURN doc'
        cursor = mdb.db.aql.execute(query)
        for data in cursor:
            #print(data)
            metadata.append(data)
        return(metadata)

In [ ]:
for i in kg:
    if i['sentence'] != '' and i['entity'][1] == 'AtLocation':
        vector = model.encode(i['sentence'])
        meta = {
                        'filename': 'none',
                        'movie_id':'none',
                        'nebula_movie_id': 'none',
                        'stage': 'none',
                        'frame_number': 'none',
                        'sentence': i,
                    }
        matloc.insert_vectors([vector.tolist()], [meta])
        

In [ ]:
for i in kg:
    if i['sentence'] != '' and i['entity'][1] == 'isFilledBy':
        vector = model.encode(i['sentence'])
        meta = {
                        'filename': 'none',
                        'movie_id':'none',
                        'nebula_movie_id': 'none',
                        'stage': 'none',
                        'frame_number': 'none',
                        'sentence': i,
                    }
        mfilled.insert_vectors([vector.tolist()], [meta])
        

In [ ]:
# for i in kg:
#     if i['sentence'] != '' and i['entity'][1] == 'isFilledBy':
#         vector = model.encode(i['sentence'])
#         print(vector.tolist())

In [ ]:
for i in kg:
    if i['sentence'] != '' and i['entity'][1] == 'isAfter':
        vector = model.encode(i['entity'][2])
        meta = {
                        'filename': 'none',
                        'movie_id':'none',
                        'nebula_movie_id': 'none',
                        'stage': 'none',
                        'frame_number': 'none',
                        'sentence': i,
                    }
        seq.insert_vectors([vector.tolist()], [meta])
        

In [ ]:
for i in kg:
    if i['sentence'] != '' and i['entity'][1] == 'isBefore':
        vector = model.encode(i['entity'][2])
        meta = {
                        'filename': 'none',
                        'movie_id':'none',
                        'nebula_movie_id': 'none',
                        'stage': 'none',
                        'frame_number': 'none',
                        'sentence': i,
                    }
        seq.insert_vectors([vector.tolist()], [meta])
        